In [ ]:
#アヤメのデータセットをロード
from sklearn.datasets import load_iris

iris = load_iris()
#print(iris.DESCR)

import pandas as pd

df = pd.DataFrame(iris.data, columns=iris.feature_names)
#df = pd.DataFrame(iris.data)
#print(df.head())

## アヤメのデータ
df['Variety'] = iris.target
df.loc[df['Variety'] == 0, 'Variety'] = 'setosa'
df.loc[df['Variety'] == 1, 'Variety'] = 'versicolor'
df.loc[df['Variety'] == 2, 'Variety'] = 'virginica'

df.describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df, hue='Variety')
plt.show()

In [ ]:
#必要なライブラリをインポート

import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch import optim

In [ ]:
#データセットをロード
iris = load_iris()
data = iris.data
label = iris.target

df = pd.DataFrame(iris.data)

#訓練用(train)と検証用(test)にデータを分ける
train_data, test_data, train_label, test_label = train_test_split(
    data, label, test_size=0.2)

print("train_data size: {}".format(len(train_data)))
print("test_data size: {}".format(len(test_data)))
print("train_label size: {}".format(len(train_label)))
print("test_label size: {}".format(len(test_label)))

In [ ]:
train_x = torch.Tensor(train_data)
test_x = torch.Tensor(test_data)
train_y = torch.LongTensor(train_label)
test_y = torch.LongTensor(test_label)
## データをテンソルのデータにセットする
train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

In [ ]:
batch_size = 5

train_batch = DataLoader(
    dataset = train_dataset,#データセットの指定
    batch_size = batch_size,#バッチサイズの指定
    shuffle = True#シャッフルするかどうかの指定
    #num_workers = 2#コア数
)
test_batch = DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False
    #num_workers = 2
)
## for文で回してサイズを確認する
for data, label in train_batch:
    print("batch data size: {}".format(data.size()))
    print("batch label size: {}".format(label.size()))
    break

In [ ]:
## ニューラルネットワークを作成
class Net(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x


In [ ]:
#モデルのインスタンス化

## パラメータ表示
#ハイパーパラメータ
D_in = 4    #入力次元
H = 100     #隠れ層次元
D_out = 3   #出力次元
epoch = 30 #学習回数
## デバイスの指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## ネットワーク実行
model = Net(D_in, H, D_out).to(device)

print("model:{}".format(model))
print("Device: {}".format(device))

In [ ]:
#損失関数の定義
# CrossEntropyLoss：交差エントロピー誤差関数

criterion = nn.CrossEntropyLoss()

#最適化関数の定義
#SGD      : optim.SGD
#Adagrad  : optim.Adagrad
#RMSprop  : optim.RMSprop
#Adadelta : optim.Adadelta
#Adam     : optim.Adam
#AdamW    : optim.AdamW

optimizer = optim.Adam(model.parameters(),lr=0.001)
#print(optimizer)

In [ ]:
# 1 epochの訓練を行う関数の定義

def train_model(model, train_loader, criterion, optimizer, device='cpu'):
    train_loss = 0       #学習損失
    train_accuracy = 0   #学習データの正答数
    
    #学習モードに設定
    model.train()
    #ミニバッチごとにデータをロードして学習
    for data, label in train_batch:
        data = data.to(device)
        label = label.to(device)
        #勾配を初期化
        optimizer.zero_grad()
        
        #データを入力して予測値を計算
        y_pred_prob = model(data)
        
        #損失の計算
        loss = criterion(y_pred_prob, label)
        
        #誤差逆伝播(勾配を計算)
        loss.backward()
        
        #パラメータの更新
        optimizer.step()
        
        #ミニバッチごとの損失を蓄積
        train_loss += loss.item()
        
        y_pred_label = torch.max(y_pred_prob, 1)[1]
        #ミニバッチごとに正解したラベル数をカウント
        train_accuracy += torch.sum(y_pred_label == label).item() / len(label)
    
    #ミニバッチの平均の損失と正解率を計算
    batch_train_loss = train_loss / len(train_batch)
    batch_train_accuracy = train_accuracy / len(train_batch)
       
    return batch_train_loss, batch_train_accuracy


In [ ]:
def test_model(model, test_loader, criterion, optimizer, device='cpu'):
    
    test_loss = 0        #評価損失
    test_accuracy = 0    #テストデータの正答数
    
    model.eval()
    #評価時に自動微分をゼロにする
    with torch.no_grad():
        for data, label in test_batch:
            data = data.to(device)
            label = label.to(device)
            #データを入力して予測値を計算
            y_pred_prob = model(data)
            #損失を計算
            loss = criterion(y_pred_prob, label)
            #ミニバッチごとの損失を備蓄
            test_loss += loss.item()
            #予測したラベルを予測確率から計算
            y_pred_label = torch.max(y_pred_prob, 1)[1]
            #ミニバッチごとに正解したラベル数をカウント
            test_accuracy += torch.sum(y_pred_label == label).item() / len(label)
    #ミニバッチの平均の損失と正解率を計算
    batch_test_loss = test_loss / len(test_batch)
    batch_test_accuracy = test_accuracy / len(test_batch)
    
    return batch_test_loss, batch_test_accuracy

In [ ]:
def lerning(model, train_loader, test_loader, criterion, opimizer, num_epochs, device='cpu'):

    ## 学習に必要な空リストを作成
    train_loss_list = []     #学習損失
    train_accuracy_list = [] #学習データ正解率
    test_loss_list = []      #評価損失
    test_accuracy_list = []  #テストデータの正答率

    # epoch数分繰り返す
    for epoch in range(1, num_epochs+1, 1):

        train_loss = train_model(model, train_loader, criterion, optimizer, device=device)
        test_loss = test_model(model, test_loader, criterion, optimizer, device=device)
        
        #print("epoch : {}, train_loss : {:.5f}, test_loss : {:.5f}" .format(epoch, train_loss, test_loss))
        
        batch_train_loss, batch_train_accuracy = train_loss
        batch_test_loss, batch_test_accuracy = test_loss
        
        print("--------")
        print("Epoch: {}/{}".format(epoch,num_epochs))      
        print("Train_Loss: {:.4f} Train_Accuracy: {:.4f}".format(batch_train_loss, batch_train_accuracy))
        print("Test_Loss: {:.4f} Test_Accuracy: {:.4f}".format(batch_test_loss, batch_test_accuracy))
        
        train_loss_list.append(batch_train_loss)
        train_accuracy_list.append(batch_train_accuracy)
        test_loss_list.append(batch_test_loss)      
        test_accuracy_list.append(batch_test_accuracy)      
    
    return train_loss_list, train_accuracy_list,test_loss_list,test_accuracy_list

In [ ]:
num_epochs = 30
train_loss_list, train_accuracy_list,test_loss_list,test_accuracy_list = lerning(model, train_batch, test_batch, criterion, optimizer, num_epochs, device=device)

In [ ]:
plt.title('Train and Test Loss')
plt.plot(range(len(train_loss_list)), train_loss_list, c='b',linestyle='-', label='train loss')
plt.plot(range(len(test_loss_list)), test_loss_list, c='r', linestyle='--',label='test loss')
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()
plt.grid()
plt.show()

plt.title('Train and Test Accuracy')
plt.plot(range(len(train_loss_list)), train_accuracy_list, c='b',linestyle='-', label='train loss')
plt.plot(range(len(test_loss_list)), test_accuracy_list, c='r', linestyle='--',label='test loss')
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend()
plt.grid()
plt.show()